# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   994.941492  0.645911  1234  995.587402  994.295581  1991.174805   
6   509.750948  0.137529  1234  509.888477  509.613420  1019.776954   
7   828.138883 -0.018922  1234  828.119961  828.157804  1656.239922   
8   849.491535  0.890195  1234  850.381730  848.601339  1700.763459   
9   187.093210 -0.319340  1234  186.773870  187.412550   373.547739   
10  677.146251 -0.793202  1234  676.353049  677.939454  1352.706098   
11  267.487462 -0.548764  1234  266.938698  268.036225   533.877396   
12  689.645945 -0.158478  1234  689.487467  689.804422  1378.974934   
13  323.122931  0.938701  1234  324.061632  322.184230   648.123264   
14  628.817379  0.994616  1234  629.811995  627.822763  1259.623990   

              f  
5   1988.591162  
6   1019.226839  
7   1656.315608  
8   1697.202679  
9    374.825100  
10  1355.878907  
11   536.072451  
12  1379.608845  
13   644.368460  
14  1255.645526

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -995.587402   994.295581
6   -509.888477   509.613420
7   -828.119961   828.157804
8   -850.381730   848.601339
9   -186.773870   187.412550
10  -676.353049   677.939454
11  -266.938698   268.036225
12  -689.487467   689.804422
13  -324.061632   322.184230
14  -629.811995   627.822763

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -995.587402   994.295581
6   -509.888477   509.613420
7   -828.119961   828.157804
8   -850.381730   848.601339
9   -186.773870   187.412550
10  -676.353049   677.939454
11  -266.938698   268.036225
12  -689.487467   689.804422
13  -324.061632   322.184230
14  -629.811995   627.822763

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-995.58740232,  994.29558082],
       [-509.88847711,  509.61341956],
       [-828.1199611 ,  828.15780418],
       [-850.3817297 ,  848.6013393 ],
       [-186.77386967,  187.41254979],
       [-676.35304908,  677.93945353],
       [-266.9386982 ,  268.03622547],
       [-689.48746709,  689.80442232],
       [-324.06163192,  322.18422984],
       [-629.8119951 ,  627.8227628 ]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1989.174805  -1988.591162
6    1017.776954  -1019.226839
7    1654.239922  -1656.315608
8    1698.763459  -1697.202679
9     371.547739   -374.825100
10   1350.706098  -1355.878907
11    531.877396   -536.072451
12   1376.974934  -1379.608845
13    646.123264   -644.368460
14   1257.623990  -1255.645526

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf